In [10]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import pandas as pd
import numpy as np
import itertools
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as pit
from matplotlib.pylab import rcParams
%matplotlib inline
from matplotlib import style
style.use('ggplot')
rcParams['figure.figsize'] = 20,10
import re

In [11]:

dfPairs = pd.DataFrame()
dfPairs = pd.read_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_5_K_Means_cluster_3.csv')
ListOfPairsToCheck = dfPairs['Pairs'].tolist()

In [12]:
df = pd.DataFrame()
df = pd.read_csv('C:/Users/archana.parihar/Downloads/allStocks16-17-18.csv')
ListOfSnPCompanies = df['Name'].tolist()
ListOfSnPCompanies = list(set(ListOfSnPCompanies))


In [13]:
for i in range(0,len(ListOfSnPCompanies)):
    if ListOfSnPCompanies[i]=='BRK.B':
        ListOfSnPCompanies[i] = 'BRKB'
    if ListOfSnPCompanies[i]=='BF.B':
        ListOfSnPCompanies[i] = 'BFB'


In [14]:
dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""
count = 0

#Making the dat dictionary 
for c in ListOfSnPCompanies:
    dfcomp = df.loc[df['Name'] == c]
    listComp = dfcomp['close'].values
    dat[c] = listComp



In [15]:
listOfPairsFinal = []
for pair in ListOfPairsToCheck:
    pair = pair.replace('(',"")
    pair = pair.replace(')',"")
    pair = pair.replace(" ", "")
    pair = pair.split(',')
    pair[0] = re.sub('[^A-Za-z0-9]+', '', pair[0])    
    pair[1] = re.sub('[^A-Za-z0-9]+', '', pair[1])
    listOfPairsFinal.append(pair)


In [16]:
import csv

count = 0
#s_and_p = 
#dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""

for pair in listOfPairsFinal:
    print(pair)
    in_market = 0
    short_pos = 0
    equity_curve = float(100)
    ret = []
    f_ret = []
    a_ret = []
    b_ret = []
    zdiff = []
    signal = [] #shod i long or short
    lCap5 = []
    sCap5 =[]
    age = [0]   #should be -1 indexed
    aged = []
    pos = [0]   #should be -1 indexed .. signal was +1 but long cap has been reached then pos = 0 .. im not in market
    for i in range (5,212):
        a_ret.append((dat[pair[0]][i]-dat[pair[0]][i-5])/dat[pair[0]][i-5])
        b_ret.append((dat[pair[1]][i]-dat[pair[1]][i-5])/dat[pair[1]][i-5])
    for j in range (59,206):
        zdiff.append(a_ret[j+1]/np.std(a_ret[0:j+1])-b_ret[j+1]/np.std(b_ret[0:j+1]))
    for k in range (65,211):
        f_ret.append((dat[pair[0]][k+1]-dat[pair[0]][k])/dat[pair[0]][k]-(dat[pair[1]][k+1]-dat[pair[1]][k])/dat[pair[1]][k])
    for l in range (0,len(zdiff)):
        if zdiff[l] < longEntryThr:
            signal.append(1)
        elif zdiff[l] > shortEntryThr:
            signal.append(-1)
        else:
            signal.append(0)

        if pos[l] == 1 and zdiff[l] > longCap:
            lCap5.append(1)
        else:
            lCap5.append(0)
        if pos[l] == -1 and zdiff[l] < shortCap:
            sCap5.append(1)
        else:
            sCap5.append(0)

        if signal[l]!=0:
            if pos[l] == 0:
                age.append(1)
            else:
                if pos[l]+signal[l] == 0:
                    age.append(1)
                else:
                    #check here if age should be age + 1
                    age.append(1)
        else:
            if age[l] == 5:
                age.append(0)
            else:
                if pos[l] == 1 and lCap5[l] != 0 or pos[l] == -1 and sCap5[l] != 0:
                    age.append(0)
                else:
                    if signal[l] + pos[l] != 0:
                        age.append(age[l] + 1)
                    else:
                        age.append(0)

        if age[l] == 5:
            if signal[l] == 0:
                aged.append(1)
            else:
                aged.append(0)
        else:
            aged.append(0)

        if signal[l] == 0:
            if aged[l] + sCap5[l] + lCap5[l] == 0:
                pos.append(pos[l])
            else:
                if aged[l] + sCap5[l] > 0 and pos[l] == -1:
                    pos.append(0)
                elif aged[l] + lCap5[l] > 0 and pos[l] == 1:
                    pos.append(0)
                else:
                    pos.append(pos[l])
        else:
            if signal[l]!=pos[l]:
                pos.append(signal[l])
            else:
                if aged[l] + sCap5[l] + lCap5[l] == 0:
                    pos.append(signal[l])
                else:
                    if aged[l] + sCap5[l] > 0 and pos[l] == -1  and signal[l] == -1:
                        pos.append(0)
                    elif aged[l] + lCap5[l] > 0 and pos[l] == 1 and signal[l] == 1:
                        pos.append(0)
                    else:
                        pos.append(signal[l])
    count = 0
        
    countOfPosResults = 0
    countOfNegResults = 0
    countOfTotalResults = 0
    
    for m in range(1,len(pos)-1):
        if(pos[m]*f_ret[m-1]>0):
            count = count + 1
        elif (pos[m]*f_ret[m-1]<0):
            count = count -1
        ret.append(pos[m]*f_ret[m-1])
        equity_curve = equity_curve*(1+pos[m]*f_ret[m-1])
        if pos[m]!=0:
            in_market = in_market + 1
        if pos[m]==-1:
            short_pos = short_pos + 1
    details = {}
    details["Sharpe"] = np.sqrt(260)*np.mean(ret)/np.std(ret)
    details["CumRet"] = (equity_curve-100)/100
    details["In_market"] = in_market/440
    details["Short"] = short_pos/440
    done_pair.append((pair[0], pair[1], details))

    if details["Sharpe"]>0 and details["CumRet"]>0:
        pos_result = pos_result  + str(done_pair[-1]) + "\n\n"
    elif details["Sharpe"]<0 and details["CumRet"]<0:
        neg_result = neg_result  + str(done_pair[-1]) + "\n\n"
    else:
        mixed_result = mixed_result  + str(done_pair[-1]) + "\n\n"
    result = result + str(done_pair[-1]) + "\n\n"

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/analysis_5_Mean_Cluster_3.txt', 'w')
file.write(result)
file.close()

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/pos_analysis_5_Mean_Cluster_3.txt', 'w')
file.write(pos_result)
file.close()

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/neg_analysis_5_Mean_Cluster_3.txt', 'w')
file.write(neg_result)
file.close()

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/mixed_analysis_5_Mean_Cluster_3.txt', 'w')
file.write(mixed_result)
file.close()



['A', 'AMD']
['A', 'APA']
['A', 'DISCK']
['A', 'EA']
['A', 'EW']
['A', 'FLR']
['A', 'FLS']
['A', 'MGM']
['A', 'NBL']
['A', 'NCLH']
['A', 'PXD']
['A', 'TGT']
['A', 'TRIP']
['AAL', 'HII']
['AAL', 'KORS']
['AAL', 'MU']
['AAP', 'ALK']
['AAP', 'AMD']
['AAP', 'APA']
['AAP', 'CMG']
['AAP', 'EXPE']
['AAP', 'ILMN']
['AAP', 'M']
['AAP', 'NEM']
['AAP', 'RCL']
['AAP', 'STX']
['AAP', 'UAL']
['AAP', 'VFC']
['AAP', 'WDC']
['ABBV', 'ABC']
['ABBV', 'ABT']
['ABBV', 'ACN']
['ABBV', 'ADI']
['ABBV', 'ADM']
['ABBV', 'ADS']
['ABBV', 'AEE']
['ABBV', 'AEP']
['ABBV', 'AES']
['ABBV', 'AET']
['ABBV', 'AFL']
['ABBV', 'AGN']
['ABBV', 'AIG']
['ABBV', 'AIZ']
['ABBV', 'AJG']
['ABBV', 'ALB']
['ABBV', 'ALL']
['ABBV', 'ALXN']
['ABBV', 'AMD']
['ABBV', 'AME']
['ABBV', 'AMG']
['ABBV', 'AMGN']
['ABBV', 'AMP']
['ABBV', 'AMT']
['ABBV', 'ANSS']
['ABBV', 'ANTM']
['ABBV', 'AON']
['ABBV', 'APA']
['ABBV', 'APC']
['ABBV', 'APD']
['ABBV', 'APH']
['ABBV', 'ARE']
['ABBV', 'ATVI']
['ABBV', 'AVB']
['ABBV', 'AVGO']
['ABBV', 'AVY']
['ABBV'

['ABT', 'HUM']
['ABT', 'IBM']
['ABT', 'IDXX']
['ABT', 'INTU']
['ABT', 'IP']
['ABT', 'IRM']
['ABT', 'ISRG']
['ABT', 'IT']
['ABT', 'ITW']
['ABT', 'IVZ']
['ABT', 'JBHT']
['ABT', 'JCI']
['ABT', 'JEC']
['ABT', 'JNJ']
['ABT', 'JPM']
['ABT', 'K']
['ABT', 'KEY']
['ABT', 'KIM']
['ABT', 'KLAC']
['ABT', 'KMB']
['ABT', 'KMX']
['ABT', 'KO']
['ABT', 'KORS']
['ABT', 'KSU']
['ABT', 'L']
['ABT', 'LB']
['ABT', 'LEG']
['ABT', 'LEN']
['ABT', 'LH']
['ABT', 'LKQ']
['ABT', 'LLY']
['ABT', 'LMT']
['ABT', 'LNC']
['ABT', 'LNT']
['ABT', 'MAA']
['ABT', 'MAT']
['ABT', 'MCD']
['ABT', 'MCHP']
['ABT', 'MCO']
['ABT', 'MDLZ']
['ABT', 'MDT']
['ABT', 'MET']
['ABT', 'MGM']
['ABT', 'MKC']
['ABT', 'MMC']
['ABT', 'MNST']
['ABT', 'MO']
['ABT', 'MON']
['ABT', 'MPC']
['ABT', 'MRK']
['ABT', 'MS']
['ABT', 'MSI']
['ABT', 'MTB']
['ABT', 'MTD']
['ABT', 'NBL']
['ABT', 'NCLH']
['ABT', 'NDAQ']
['ABT', 'NEE']
['ABT', 'NEM']
['ABT', 'NKE']
['ABT', 'NTAP']
['ABT', 'NTRS']
['ABT', 'NUE']
['ABT', 'NVDA']
['ABT', 'NWL']
['ABT', 'OKE']
['ABT',

['ADBE', 'AEP']
['ADBE', 'AES']
['ADBE', 'AET']
['ADBE', 'AFL']
['ADBE', 'AGN']
['ADBE', 'AIG']
['ADBE', 'AIZ']
['ADBE', 'AJG']
['ADBE', 'ALB']
['ADBE', 'ALK']
['ADBE', 'ALL']
['ADBE', 'ALXN']
['ADBE', 'AMD']
['ADBE', 'AME']
['ADBE', 'AMG']
['ADBE', 'AMGN']
['ADBE', 'AMP']
['ADBE', 'AMT']
['ADBE', 'ANDV']
['ADBE', 'ANSS']
['ADBE', 'ANTM']
['ADBE', 'AON']
['ADBE', 'AOS']
['ADBE', 'APA']
['ADBE', 'APC']
['ADBE', 'APD']
['ADBE', 'APH']
['ADBE', 'ARE']
['ADBE', 'ARNC']
['ADBE', 'AVB']
['ADBE', 'AVGO']
['ADBE', 'AVY']
['ADBE', 'AWK']
['ADBE', 'AXP']
['ADBE', 'AYI']
['ADBE', 'AZO']
['ADBE', 'BAC']
['ADBE', 'BAX']
['ADBE', 'BBT']
['ADBE', 'BBY']
['ADBE', 'BDX']
['ADBE', 'BEN']
['ADBE', 'BFB']
['ADBE', 'BIIB']
['ADBE', 'BK']
['ADBE', 'BLK']
['ADBE', 'BLL']
['ADBE', 'BWA']
['ADBE', 'BXP']
['ADBE', 'C']
['ADBE', 'CA']
['ADBE', 'CAG']
['ADBE', 'CAH']
['ADBE', 'CAT']
['ADBE', 'CB']
['ADBE', 'CBOE']
['ADBE', 'CBS']
['ADBE', 'CCL']
['ADBE', 'CERN']
['ADBE', 'CF']
['ADBE', 'CHD']
['ADBE', 'CHRW']
['A

['ADI', 'WEC']
['ADI', 'WHR']
['ADI', 'WMT']
['ADI', 'WU']
['ADI', 'XEL']
['ADI', 'YUM']
['ADI', 'ZBH']
['ADI', 'ZION']
['ADM', 'AES']
['ADM', 'AKAM']
['ADM', 'AMG']
['ADM', 'AMP']
['ADM', 'AVGO']
['ADM', 'BEN']
['ADM', 'BIIB']
['ADM', 'CF']
['ADM', 'CMG']
['ADM', 'CMI']
['ADM', 'COG']
['ADM', 'COO']
['ADM', 'DVN']
['ADM', 'EOG']
['ADM', 'EQT']
['ADM', 'FDX']
['ADM', 'FFIV']
['ADM', 'GILD']
['ADM', 'GPS']
['ADM', 'HBI']
['ADM', 'HCA']
['ADM', 'HES']
['ADM', 'HST']
['ADM', 'HUM']
['ADM', 'ILMN']
['ADM', 'JWN']
['ADM', 'MCD']
['ADM', 'MCK']
['ADM', 'MS']
['ADM', 'NVDA']
['ADM', 'OKE']
['ADM', 'PRGO']
['ADM', 'PVH']
['ADM', 'PWR']
['ADM', 'RRC']
['ADM', 'SPGI']
['ADM', 'STX']
['ADM', 'SWKS']
['ADM', 'TWX']
['ADM', 'UHS']
['ADM', 'WDC']
['ADM', 'XEC']
['ADP', 'AEE']
['ADP', 'AEP']
['ADP', 'AES']
['ADP', 'AET']
['ADP', 'AFL']
['ADP', 'AGN']
['ADP', 'AIG']
['ADP', 'AJG']
['ADP', 'ALB']
['ADP', 'ALK']
['ADP', 'ALL']
['ADP', 'ALXN']
['ADP', 'AME']
['ADP', 'AMT']
['ADP', 'ANDV']
['ADP', 'ANSS']

['AEE', 'PNW']
['AEE', 'PRGO']
['AEE', 'PRU']
['AEE', 'PSA']
['AEE', 'PVH']
['AEE', 'PWR']
['AEE', 'PXD']
['AEE', 'RCL']
['AEE', 'REG']
['AEE', 'RF']
['AEE', 'ROK']
['AEE', 'RTN']
['AEE', 'SBUX']
['AEE', 'SJM']
['AEE', 'SLB']
['AEE', 'SLG']
['AEE', 'SO']
['AEE', 'SPG']
['AEE', 'STT']
['AEE', 'STZ']
['AEE', 'SWK']
['AEE', 'SYMC']
['AEE', 'SYY']
['AEE', 'T']
['AEE', 'TAP']
['AEE', 'TDG']
['AEE', 'TIF']
['AEE', 'TJX']
['AEE', 'TMK']
['AEE', 'TMO']
['AEE', 'TRIP']
['AEE', 'TROW']
['AEE', 'TRV']
['AEE', 'TSN']
['AEE', 'TWX']
['AEE', 'TXN']
['AEE', 'TXT']
['AEE', 'UAL']
['AEE', 'UHS']
['AEE', 'UNM']
['AEE', 'UPS']
['AEE', 'V']
['AEE', 'VNO']
['AEE', 'VRSK']
['AEE', 'VZ']
['AEE', 'WEC']
['AEE', 'WMT']
['AEE', 'WU']
['AEE', 'XEC']
['AEE', 'XRAY']
['AEE', 'XYL']
['AEE', 'YUM']
['AEE', 'ZBH']
['AEE', 'ZION']
['AEP', 'AES']
['AEP', 'AET']
['AEP', 'AFL']
['AEP', 'AGN']
['AEP', 'AIG']
['AEP', 'AIZ']
['AEP', 'AJG']
['AEP', 'AKAM']
['AEP', 'ALB']
['AEP', 'ALK']
['AEP', 'ALL']
['AEP', 'ALXN']
['AEP', 

...

['PRGO', 'SEE']
['PRGO', 'STX']
['PRGO', 'VFC']
['PRGO', 'WDC']
['PRGO', 'YUM']
['PRU', 'PSA']
['PRU', 'PSX']
['PRU', 'PVH']
['PRU', 'PWR']
['PRU', 'PX']
['PRU', 'QCOM']
['PRU', 'RCL']
['PRU', 'RE']
['PRU', 'REG']
['PRU', 'REGN']
['PRU', 'RHI']
['PRU', 'RHT']
['PRU', 'RJF']
['PRU', 'RL']
['PRU', 'RMD']
['PRU', 'ROK']
['PRU', 'ROP']
['PRU', 'ROST']
['PRU', 'RRC']
['PRU', 'RSG']
['PRU', 'RTN']
['PRU', 'SBUX']
['PRU', 'SCG']
['PRU', 'SCHW']
['PRU', 'SEE']
['PRU', 'SHW']
['PRU', 'SJM']
['PRU', 'SLB']
['PRU', 'SLG']
['PRU', 'SNA']
['PRU', 'SNPS']
['PRU', 'SO']
['PRU', 'SPG']
['PRU', 'SPGI']
['PRU', 'SRCL']
['PRU', 'SRE']
['PRU', 'STT']
['PRU', 'STZ']
['PRU', 'SWK']
['PRU', 'SYK']
['PRU', 'SYMC']
['PRU', 'SYY']
['PRU', 'T']
['PRU', 'TAP']
['PRU', 'TDG']
['PRU', 'TEL']
['PRU', 'TGT']
['PRU', 'TIF']
['PRU', 'TJX']
['PRU', 'TMK']
['PRU', 'TMO']
['PRU', 'TPR']
['PRU', 'TRIP']
['PRU', 'TROW']
['PRU', 'TRV']
['PRU', 'TSCO']
['PRU', 'TSN']
['PRU', 'TSS']
['PRU', 'TWX']
['PRU', 'TXN']
['PRU', 'TXT']

['RE', 'USB']
['RE', 'UTX']
['RE', 'V']
['RE', 'VAR']
['RE', 'VNO']
['RE', 'VRSK']
['RE', 'VTR']
['RE', 'VZ']
['RE', 'WFC']
['RE', 'WM']
['RE', 'WMT']
['RE', 'WU']
['RE', 'WY']
['RE', 'WYNN']
['RE', 'XEL']
['RE', 'YUM']
['RE', 'ZBH']
['RE', 'ZION']
['REG', 'RF']
['REG', 'ROK']
['REG', 'SCG']
['REG', 'SHW']
['REG', 'SLB']
['REG', 'SO']
['REG', 'TAP']
['REG', 'TIF']
['REG', 'TRIP']
['REG', 'TRV']
['REG', 'TXN']
['REG', 'UAL']
['REG', 'UHS']
['REG', 'UNM']
['REG', 'VRTX']
['REG', 'VTR']
['REG', 'WMB']
['REG', 'XEC']
['REG', 'XEL']
['REG', 'YUM']
['REG', 'ZION']
['REGN', 'RF']
['REGN', 'RHI']
['REGN', 'RHT']
['REGN', 'RJF']
['REGN', 'RL']
['REGN', 'RMD']
['REGN', 'ROK']
['REGN', 'ROP']
['REGN', 'ROST']
['REGN', 'RRC']
['REGN', 'RSG']
['REGN', 'RTN']
['REGN', 'SBUX']
['REGN', 'SCG']
['REGN', 'SCHW']
['REGN', 'SEE']
['REGN', 'SHW']
['REGN', 'SJM']
['REGN', 'SLB']
['REGN', 'SLG']
['REGN', 'SNA']
['REGN', 'SNPS']
['REGN', 'SO']
['REGN', 'SPG']
['REGN', 'SPGI']
['REGN', 'SRCL']
['REGN', 'SRE']


['SCHW', 'XYL']
['SCHW', 'YUM']
['SCHW', 'ZBH']
['SCHW', 'ZION']
['SCHW', 'ZTS']
['SEE', 'TSCO']
['SEE', 'UHS']
['SEE', 'UNH']
['SEE', 'VFC']
['SEE', 'WBA']
['SEE', 'WDC']
['SEE', 'WHR']
['SHW', 'SLG']
['SHW', 'TDG']
['SHW', 'TGT']
['SHW', 'TPR']
['SHW', 'TXN']
['SHW', 'TXT']
['SHW', 'UAL']
['SHW', 'UNH']
['SHW', 'UTX']
['SHW', 'VAR']
['SHW', 'VRTX']
['SHW', 'VTR']
['SHW', 'WBA']
['SHW', 'WHR']
['SHW', 'WMT']
['SHW', 'WY']
['SHW', 'ZBH']
['SHW', 'ZTS']
['SJM', 'SO']
['SJM', 'STX']
['SJM', 'SWK']
['SJM', 'SYMC']
['SJM', 'SYY']
['SJM', 'TDG']
['SJM', 'TIF']
['SJM', 'TMO']
['SJM', 'TSN']
['SJM', 'TXT']
['SJM', 'UAL']
['SJM', 'UNP']
['SJM', 'UPS']
['SJM', 'URI']
['SJM', 'VFC']
['SJM', 'VLO']
['SJM', 'VRSK']
['SJM', 'VTR']
['SJM', 'WDC']
['SJM', 'WM']
['SJM', 'WMB']
['SJM', 'WMT']
['SJM', 'WU']
['SJM', 'WY']
['SJM', 'XL']
['SJM', 'XYL']
['SJM', 'ZION']
['SLB', 'TRIP']
['SLB', 'UAL']
['SLB', 'VIAB']
['SLG', 'TEL']
['SLG', 'TXN']
['SLG', 'TXT']
['SLG', 'UAL']
['SLG', 'UNP']
['SLG', 'VTR']
['S

['SYK', 'SYY']
['SYK', 'T']
['SYK', 'TDG']
['SYK', 'TGT']
['SYK', 'TIF']
['SYK', 'TJX']
['SYK', 'TMK']
['SYK', 'TPR']
['SYK', 'TROW']
['SYK', 'TWX']
['SYK', 'TXT']
['SYK', 'UAL']
['SYK', 'UNH']
['SYK', 'UNM']
['SYK', 'UNP']
['SYK', 'UPS']
['SYK', 'USB']
['SYK', 'UTX']
['SYK', 'V']
['SYK', 'VAR']
['SYK', 'VFC']
['SYK', 'VIAB']
['SYK', 'VMC']
['SYK', 'VNO']
['SYK', 'VRSK']
['SYK', 'VRTX']
['SYK', 'VTR']
['SYK', 'VZ']
['SYK', 'WBA']
['SYK', 'WEC']
['SYK', 'WFC']
['SYK', 'WM']
['SYK', 'WMT']
['SYK', 'WU']
['SYK', 'WY']
['SYK', 'WYNN']
['SYK', 'XEL']
['SYK', 'XL']
['SYK', 'XOM']
['SYK', 'YUM']
['SYK', 'ZION']
['SYMC', 'SYY']
['SYMC', 'T']
['SYMC', 'TAP']
['SYMC', 'TDG']
['SYMC', 'TEL']
['SYMC', 'TGT']
['SYMC', 'TJX']
['SYMC', 'TMK']
['SYMC', 'TMO']
['SYMC', 'TRV']
['SYMC', 'TSCO']
['SYMC', 'TSN']
['SYMC', 'TSS']
['SYMC', 'TWX']
['SYMC', 'TXT']
['SYMC', 'UAL']
['SYMC', 'UHS']
['SYMC', 'ULTA']
['SYMC', 'UNH']
['SYMC', 'UNM']
['SYMC', 'UNP']
['SYMC', 'UPS']
['SYMC', 'USB']
['SYMC', 'UTX']
['SY

['UAL', 'VRSK']
['UAL', 'VRTX']
['UAL', 'VTR']
['UAL', 'VZ']
['UAL', 'WAT']
['UAL', 'WBA']
['UAL', 'WDC']
['UAL', 'WEC']
['UAL', 'WFC']
['UAL', 'WHR']
['UAL', 'WM']
['UAL', 'WMB']
['UAL', 'WMT']
['UAL', 'WU']
['UAL', 'WY']
['UAL', 'WYNN']
['UAL', 'XEC']
['UAL', 'XEL']
['UAL', 'XL']
['UAL', 'XLNX']
['UAL', 'XOM']
['UAL', 'XRAY']
['UAL', 'XRX']
['UAL', 'XYL']
['UAL', 'YUM']
['UAL', 'ZBH']
['UAL', 'ZION']
['UAL', 'ZTS']
['UHS', 'XEC']
['UHS', 'XLNX']
['ULTA', 'URI']
['ULTA', 'WYNN']
['UNH', 'UNM']
['UNH', 'UNP']
['UNH', 'UPS']
['UNH', 'URI']
['UNH', 'USB']
['UNH', 'UTX']
['UNH', 'V']
['UNH', 'VAR']
['UNH', 'VFC']
['UNH', 'VIAB']
['UNH', 'VLO']
['UNH', 'VMC']
['UNH', 'VNO']
['UNH', 'VRSK']
['UNH', 'VRTX']
['UNH', 'VTR']
['UNH', 'VZ']
['UNH', 'WAT']
['UNH', 'WBA']
['UNH', 'WEC']
['UNH', 'WFC']
['UNH', 'WHR']
['UNH', 'WM']
['UNH', 'WMB']
['UNH', 'WMT']
['UNH', 'WU']
['UNH', 'WY']
['UNH', 'WYNN']
['UNH', 'XEC']
['UNH', 'XEL']
['UNH', 'XL']
['UNH', 'XLNX']
['UNH', 'XOM']
['UNH', 'XRAY']
['UNH'

In [17]:

listOfPosResult = pos_result.split('\n\n')
print(len(listOfPosResult))

listOfResult = result.split('\n\n')
print(len(listOfResult))

success = (len(listOfPosResult)/len(listOfResult))*100
print('Percentage of profiting pairs in Cluster: '+ str(success)+'%')

17798
39104
Percentage of profiting pairs in Cluster: 45.51452536824877%


In [18]:
list11 =[]
list22 = []
listRet = []
for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    
    list11.append(list1[0])
    list22.append(list1[1])
    listRet.append(float(cumR))
    
maxVal = max(listRet)

for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    cumR = float(cumR)
    
    if(cumR==maxVal):
        print(list1[0],list1[1],round(maxVal,2))

NWL SCG 2.51
